In [1]:
import boto3
import time
import os
from dotenv import load_dotenv

In [2]:
# Load environment variables from .env file
load_dotenv()

# Initialize AWS credentials from the .env file
AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
AWS_SESSION_TOKEN = os.getenv('AWS_SESSION_TOKEN')
AWS_REGION = os.getenv('AWS_REGION')

In [3]:
# Initialize boto3 client for Kinesis with your credentials
kinesis_client = boto3.client(
    'kinesis',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    aws_session_token=AWS_SESSION_TOKEN,
    region_name=AWS_REGION
)

In [4]:
# Function to create a Kinesis Data Stream
def create_kinesis_stream(stream_name, shard_count):
    try:
        response = kinesis_client.create_stream(
            StreamName=stream_name,
            ShardCount=shard_count  # Number of shards, adjust according to your needs
        )
        print(f"Kinesis Stream '{stream_name}' created successfully!")
    except kinesis_client.exceptions.ResourceInUseException:
        print(f"Stream '{stream_name}' already exists.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Function to check stream status
def wait_for_stream_active(stream_name):
    while True:
        response = kinesis_client.describe_stream(StreamName=stream_name)
        stream_status = response['StreamDescription']['StreamStatus']
        print(f"Current stream status: {stream_status}")
        
        if stream_status == 'ACTIVE':
            print(f"Stream '{stream_name}' is now active.")
            break
        time.sleep(10)

# Define the stream name and shard count
stream_name = 'OrderStreamForDispatching'  # Replace with your stream name
shard_count = 1  # Set the number of shards (you can increase this based on throughput)

# Create the Kinesis stream
create_kinesis_stream(stream_name, shard_count)

# Wait for the stream to become active
wait_for_stream_active(stream_name)


Stream 'OrderStreamForDispatching' already exists.
Current stream status: ACTIVE
Stream 'OrderStreamForDispatching' is now active.
